#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [17]:
# Configurer pandas pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head(10)

C:\Users\icarr\AppData\Local\Temp\ipykernel_5260\4010832920.py:5: DtypeWarning: Columns (6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.0,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.0,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.0,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.0,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225
5,1771789139,Room For Growth!,Couture,Fashion,USD,2016-05-02,2000.0,2016-04-11 18:15:00,6.0,failed,2,US,6.0,6.000000
6,1301627822,Build a mini Udemy with Laravel and Vuejs,Web,Technology,CAD,2018-01-11,3000.0,2017-12-12 01:37:26,155.0,live,7,CA,15.563996,123.811806
7,881336601,Applitizer,Software,Technology,HKD,2017-08-20,100000.0,2017-07-21 16:50:05,1000.0,failed,1,HK,128.08538,127.831467
8,1669374237,"Help I Am Wolves fund their first album ""ABCD""...",Music,Music,EUR,2017-12-04,1000.0,2017-10-27 16:55:47,1465.0,successful,46,BE,0.0,1720.210418
9,556821416,Eco Bamboo Underwear funding....GOT WOOD ?,Fashion,Fashion,GBP,2012-12-13,9500.0,2012-11-13 10:58:34,1212.0,failed,43,GB,1928.137119,1953.672808


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [22]:
# Afficher les colonnes 6, 8, 10, 12 (celles avec warnings)
print("Colonnes avec warnings de type:")
print(df_ks.columns[[6,8,10,12]].tolist())
print("\nToutes les colonnes:")
print(df_ks.columns.tolist())

Colonnes avec warnings de type:
['goal', 'pledged', 'backers', 'usd pledged']

Toutes les colonnes:
['ID', 'name', 'category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged', 'usd_pledged_real']


## Question 0

### Netoyer les données

In [25]:
# Nettoyage de données
# Au lieu de supprimer toutes les lignes avec des valeurs manquantes,
# on peut soit :
# 1) Garder les colonnes importantes seulement
# 2) Remplir les valeurs manquantes
# 3) Supprimer uniquement les lignes avec trop de valeurs manquantes

# Option : Vérifier les valeurs manquantes par colonne
print("Valeurs manquantes par colonne:")
print(df_ks.isnull().sum())
print(f"\nNombre de lignes avant nettoyage: {len(df_ks)}")

# Supprimer uniquement les lignes où les colonnes essentielles sont manquantes
# (garde plus de données)
df_ks = df_ks.dropna(subset=['name', 'goal', 'pledged', 'state', 'country'])

print(f"Nombre de lignes après nettoyage: {len(df_ks)}")
print(f"Nombre de colonnes: {len(df_ks.columns)}")

Valeurs manquantes par colonne:
ID                  0
name                0
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
dtype: int64

Nombre de lignes avant nettoyage: 148516
Nombre de lignes après nettoyage: 148516
Nombre de colonnes: 14


### Importer les données

In [26]:
# Configurer pandas pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

# Check the actual column names
print("Toutes les colonnes:")
print(df_ks.columns.tolist())
print(f"\nNombre total de colonnes: {len(df_ks.columns)}")

Toutes les colonnes:
['ID', 'name', 'category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged', 'usd_pledged_real']

Nombre total de colonnes: 14


In [ ]:
# Vérifier que les colonnes existent
print("Colonnes disponibles:", df_ks.columns.tolist())

df_ks.info()

Colonnes disponibles: ['ID', 'name', 'category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged', 'usd_pledged_real']
<class 'pandas.core.frame.DataFrame'>
Index: 148516 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                148516 non-null  int64  
 1   name              148516 non-null  object 
 2   category          148516 non-null  object 
 3   main_category     148516 non-null  object 
 4   currency          148516 non-null  object 
 5   deadline          148516 non-null  object 
 6   goal              148516 non-null  object 
 7   launched          148516 non-null  object 
 8   pledged           148516 non-null  object 
 9   state             148516 non-null  object 
 10  backers           148516 non-null  object 
 11  country           148516 non-null  object 
 12  usd pledged       148516 non-n

In [36]:
# Convert 'launched' and 'deadline' columns to datetime
if 'launched' in df_ks.columns:
    df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
if 'deadline' in df_ks.columns:
    df_ks['deadline'] = pd.to_datetime(df_ks['deadline'], errors='coerce')

## Question 1  

In [38]:
#1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'], errors='coerce')
top_5_projects = df_ks.nlargest(5, 'pledged')
print("Top 5 projets ayant reçu le plus de promesse de dons:")
print(top_5_projects[['name', 'pledged']])



Top 5 projets ayant reçu le plus de promesse de dons:
                                                     name      pledged
76581   COOLEST COOLER: 21st Century Cooler that's Act...  13285226.36
99870              Pebble 2, Time 2 + All-New Pebble Core  12779843.49
79708   Expect the Unexpected. digiFilmï¿½ Camera by Y...  10035296.00
113465             OUYA: A New Kind of Video Game Console   8596474.58
144112             The Everyday Backpack, Tote, and Sling   6565782.50


## Question 2

In [39]:
#2) Compter le nombre de projets ayant atteint leur but.
successful_projects_count = df_ks[df_ks['state'] == 'successful'].shape[0]
print(f"Nombre de projets ayant atteint leur but: {successful_projects_count}")


Nombre de projets ayant atteint leur but: 52998


## Question 3

In [41]:
#3) Compter le nombre de projets pour chaque catégorie.
category_counts = df_ks['category'].value_counts()
print("Nombre de projets par catégorie:")
print(category_counts)


Nombre de projets par catégorie:
category
Product Design      8885
Documentary         6497
Tabletop Games      5579
Music               5293
Shorts              4857
                    ... 
Letterpress           24
Literary Spaces       10
Chiptune              10
Taxidermy              7
Grace is Leaving       1
Name: count, Length: 160, dtype: int64


## Question 4

In [42]:
#4) Compter le nombre de projets français ayant été instanciés avant 2016.

df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
french_projects_pre2016 = df_ks[(df_ks['country'] == 'FR') & (df_ks['launched'] < '2016-01-01')]
print(f"Nombre de projets français instanciés avant 2016: {len(french_projects_pre2016)}")

Nombre de projets français instanciés avant 2016: 330


## Question 5

In [45]:
#5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
df_ks['goal'] = pd.to_numeric(df_ks['goal'], errors='coerce')
us_projects_over_200k = df_ks[(df_ks['country'] == 'US') & (df_ks['goal'] > 200000)]
print("Projets américains ayant demandé plus de 200 000 dollars:")
print(us_projects_over_200k[['name', 'goal', 'country']])


Projets américains ayant demandé plus de 200 000 dollars:
                                                     name       goal country
162                                   A CALL TO ADVENTURE   287000.0      US
341              Storybricks, the storytelling online RPG   250000.0      US
349                                    Shine On New World   300000.0      US
364                                             Nightclub  3000000.0      US
752                                  Nastaran (Wild Rose)   250000.0      US
...                                                   ...        ...     ...
149779            BassCase - Bold Sound in an iPhone Case   350000.0      US
149785                  Back in the 50's again (Canceled)   275000.0      US
149795  D.A.'s dresses, ties, bow ties, blankets, gift...   250000.0      US
149811  ChuSingura46+1 - A Visual Novel with Moe Samur...   210000.0      US
149895                                    Metro County PD   250000.0      US

[1841 rows x 3 co

## Question 6 

In [46]:
#6) Compter le nombre de projet ayant "Sport" dans leur nom
sport_projects_count = df_ks['name'].str.contains('Sport', case=False, na=False).sum()
print(f"Nombre de projets ayant 'Sport' dans leur nom: {sport_projects_count}")

Nombre de projets ayant 'Sport' dans leur nom: 500
